In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score


In [2]:
import os
os.environ["OPENAI_API_KEY"] = ''
from openai import OpenAI
client = OpenAI()

In [3]:
import re
def get_classification(prompt, errors):
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ])
    reply = completion.choices[0].message.content
    # print(reply)
    match=re.search(r'classification::.*', reply)
    answer = [0] * len(df_train.columns.tolist()[2:])
    if match and match.group():
        try:
            answer = eval(match.group()[17:])
        except:
            errors += 1 
    return answer, errors

In [4]:
test_path = 'public_data_test/track_a/test/'
train_path = 'public_data_test/track_a/train/'

In [5]:
for filename in os.listdir(train_path):
    if not filename.endswith('.csv'):
        continue
    language = filename[0:-4]
    print(language)
    df_train = pd.read_csv(train_path+filename)
    df_test = pd.read_csv(test_path+filename)

    prompt1 = ("Hi, I have a project that I'm doing for various languages. The task description is: Given a target text snippet, " 
    "predict the perceived emotion(s) of the speaker. Specifically, select whether each of the following "
    "emotions apply: joy, sadness, fear, anger, surprise, or disgust. In other words, label the text snippet "
    "with: joy (1) or no joy (0), sadness (1) or no sadness (0), anger (1) or no anger (0), surprise (1) or no "
    "surprise (0), and disgust (1) or no disgust (0). I am going to give you a small set of training examples "
    "for you to get an understanding of what's expected. First I will tell you the set and order of emotions "
    "that are present in the training set. Then I'll give you examples. After this I will ask you to return me "
    "answers for test and they should be in the format 'classification:: []' so that I can parse the answer. "
    "The array in this response should have the same number of emotions and order of emotions as training set! ")

    emotions = df_train.columns.tolist()[2:]
    prompt2 = "The order of emotions is {}".format(emotions)
    prompt = prompt1+ "\n" +prompt2

    sample_sentence = ""
    for index in range(50):
        text = df_train.iloc[index]['text']
        classification = df_train.iloc[index][2:].tolist()
        sample_sentence += "\n Example number:: {}, text:: {}, classification:: {} ".format(index,text,classification)
    prompt = prompt+sample_sentence

    errors = 0
    result = []
    
    from tqdm import tqdm
    
    for i in tqdm(range(df_test.shape[0])):
        test_case = "\n I don't need explanations, what is the answer for the test case:: {}".format(df_test.iloc[i]['text'])
        test_prompt = prompt + test_case
        answer, errors = get_classification(test_prompt, errors)
        answer.insert(0,df_test.iloc[i]['id'])
        result.append(answer)

    print("number of errors in {} is {}".format(language,errors))

    c = df_train.columns.tolist()[2:]
    c.insert(0,'id')
    result_df = pd.DataFrame(result,columns=c)
    result_df.to_csv("track_a/pred_{}.csv".format(language),index=False)

orm


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1721/1721 [25:30<00:00,  1.12it/s]


number of errors in orm is 0
ptbr


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2226/2226 [29:58<00:00,  1.24it/s]


number of errors in ptbr is 0
ptmz


...

number of errors in pcm is 0
ron


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1119/1119 [17:32<00:00,  1.06it/s]


number of errors in ron is 0
kin


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1231/1231 [18:34<00:00,  1.10it/s]

number of errors in kin is 0
